In [ ]:
## train test data

In [ ]:
pip install optuna catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.9 MB/s eta 0:00:00


In [ ]:
def outlier_viz(col):
    '''Distribution and Boxplot for Outlier Detection by @kartik2khandelwal'''
    import random
    color = random.choice(['r', 'g', 'b'])
    fig, ax = plt.subplots(1,2,figsize=(15,5))
    sns.distplot(col, ax=ax[0], color=color)
    sns.boxplot(col, ax=ax[1], color=color)
    plt.suptitle('Distribution & Boxplot for Outlier Detection')
    fig.show()
    return None

def my_print(s):
    '''Custom print function by @kartik2khandelwal :)'''
    a = 4
    for i in s:
        a+=1
    return print('-' * a + '\n' + '| ' + s + ' |' + '\n' + '-' * a)

In [ ]:
# Mathematical Libraries needed for DataFrame
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
plt.style.use('dark_background')

# Remove unnecessary warnings
import warnings
warnings.filterwarnings('ignore')

# Display DataFrames
from IPython.display import display, display_html

# Encoding Categorical Variables
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()

# Splitting Train-Test data
from sklearn.model_selection import train_test_split

# For Cross Validation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# For showing Progress Bar
from tqdm.notebook import tqdm

# Models
import optuna
import xgboost as xgb
import lightgbm as lgbm
import catboost as catb

In [ ]:
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_datar2.csv')

In [ ]:
# test_data2 = pd.read_csv('test_data2.csv')

In [ ]:
train_data.shape
train_data.head(2)

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,winner,winner_id,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,team_count_sub20eco_last15,team_count_2wickets_last15,team_win_amount,team_winp_ground
0,9331181,Ba,11283,9373356.0:7857520.0:4232164.0:4566540.0:329940...,Hl Ph,12634,3500958.0:4231751.0:2735081.0:2035102.0:369833...,Hl Ph,12634,Hl Ph,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.0,100.0,157.178571,2.033333,1.500000,0.424528,0.504950
1,8797060,Ed,20,2089079.0:6139370.0:2076192.0:62432.0:2083409....,Wt Is,41,4690258.0:4069666.0:4230127.0:1942317.0:161392...,Ed,20,Wt Is,field,Kn Ol Bn Bs,Bridgetown,2022-01-23,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,156.0,50.0,103.500000,0.871287,0.892857,1.800000,0.009901


In [ ]:
test_data.shape
test_data.head(2)

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,team_count_sub20eco_last15,team_count_2wickets_last15,team_win_amount,team_winp_ground
0,9272619,Me Ss,33949,4003390.0:7960994.0:3901078.0:2669316.0:373710...,Ht Hs,33928,5843200.0:4223883.0:4655384.0:6249256.0:216159...,Ht Hs,field,Be Ol Ht,Hobart,2023-01-09,night match,Bg Bh Le,2022/23,6348,0.666667,1.000000,167.933333,60.0,154.115385,0.878049,1.294118,8.588235,0.673203
1,9086958,Na,209,5836452.0:8246468.0:7500324.0:3065502.0:363350...,Si La,69,7200598.0:4403531.0:3260564.0:2420760.0:239834...,Si La,field,GA Sm Sh Gg Va,Geelong,2022-10-16,day/night match,Si La tr of Aa,2022/23,3961,0.142857,0.207921,141.888889,0.0,142.833333,0.706667,1.000000,1.000000,1.000000


#### Create dependent column

In [ ]:
train_data['winner_01'] = train_data.apply(lambda x: 1 if (x['team2']==x['winner']) else 0, axis=1)

#### Pre process

In [ ]:
## Toss winner to numerical - 1 if team2 wins, else 0

train_data['toss_winner_01'] = np.where(train_data['toss winner']==train_data['team2'], 1, 0)
test_data['toss_winner_01'] = np.where(test_data['toss winner']==test_data['team2'], 1, 0)

In [ ]:
# test_data2['toss_winner_01'] = np.where(test_data2['toss winner']==test_data2['team2'], 1, 0)

In [ ]:
## Toss decision - categorical - 1 if winner bats, 0 otherwise

train_data['toss_decision_01'] = np.where(train_data['toss decision']=='bat', 1, 0)
test_data['toss_decision_01'] = np.where(test_data['toss decision']=='bat', 1, 0)

In [ ]:
# test_data2['toss_decision_01'] = np.where(test_data2['toss decision']=='bat', 1, 0)

#### Selecting relevant columns

In [ ]:
X,y = train_data[['toss_winner_01','toss_decision_01','team_count_50runs_last15','team_winp_last5','team1only_avg_runs_last15','team1_winp_team2_last15','ground_avg_runs_last15',"team_count_sub20eco_last15","team_count_2wickets_last15","team_win_amount","team_winp_ground"]], train_data['winner_01']

In [ ]:
X_test = test_data[X.columns.tolist()]
X_test.fillna(0, inplace=True)

In [ ]:
X.fillna(0, inplace=True)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=24)

In [ ]:
# dtrain = xgb.DMatrix(X_train, label=y_train)
# dvalid  = xgb.DMatrix(X_valid, label=y_valid)

# def objectivexgb(trial):

#     param = {
#         "verbosity": 0,
#         "objective": "multi:softmax",
#         "num_class": 5,
#         'max_depth': trial.suggest_int('max_depth', 2, 30),
#         'n_estimators': 30,
#         # L2 regularization weight.
#         "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
#         # L1 regularization weight.
#         "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
#         # sampling ratio for training data.
#         "subsample": trial.suggest_float("subsample", 0.6, 1.0),
#         # sampling according to each tree.
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
#         'min_child_weight': trial.suggest_int('min_child_weight', 2, 20)
#     }

#     bst = xgb.train(param, dtrain)
#     preds = bst.predict(dvalid)
#     pred_labels = np.rint(preds)
#     accuracy = accuracy_score(y_valid, pred_labels)
#     return accuracy

In [ ]:
# %%time
# study = optuna.create_study(direction="maximize")
# study.optimize(objectivexgb, n_trials=100, show_progress_bar=True)

In [ ]:
# new_paramxgb = study.best_params
# new_paramxgb
# optuna.visualization.plot_optimization_history(study)
# optuna.visualization.plot_slice(study)

In [ ]:
xgb_param={'max_depth': 15,
           "n_estimators":100,
           "eta": 0.01,
 'lambda': 9.986425773717331e-06,
 'alpha': 3.5431708869032965e-05,
 'subsample': 0.8760612839207286,
 'colsample_bytree': 0.21261134376452542,
 'min_child_weight': 13}

In [ ]:
# def objectivelgbm(trial):

#     param = {
#             'metric': 'auc',
#             'random_state': 24,
#             'n_estimators': 4000,
#             'boosting_type': trial.suggest_categorical("boosting_type", ["gbdt"]),
#             'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 10.0),
#             'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-3, 10.0),
#             'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
#             'bagging_fraction': trial.suggest_categorical('bagging_fraction', [0.6, 0.7, 0.80]),
#             'feature_fraction': trial.suggest_categorical('feature_fraction', [0.6, 0.7, 0.80]),
#             'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3 , 10),
#             'max_depth': trial.suggest_int('max_depth', 2, 20, step=1),
#             'num_leaves' : trial.suggest_int('num_leaves', 13, 148, step=5),
#             'min_child_samples': trial.suggest_int('min_child_samples', 1, 96, step=5),
#         }

#     clf = lgbm.LGBMClassifier(**param)
#     clf.fit(X_train, y_train)

#     preds = clf.predict(X_valid)
#     pred_labels = np.rint(preds)
#     accuracy = accuracy_score(y_valid, pred_labels)
#     return accuracy

In [ ]:
# %%time
# study = optuna.create_study(direction="maximize")
# study.optimize(objectivelgbm, n_trials=50, show_progress_bar=True)

In [ ]:
# new_paramlgbm = study.best_params
# new_paramlgbm
# # optuna.visualization.plot_optimization_history(study)
# # optuna.visualization.plot_slice(study)

In [ ]:
# def objectivecatb(trial):

#     param = {
#         "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
#         "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
#         "depth": trial.suggest_int("depth", 1, 20),
#         "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
#         "bootstrap_type": trial.suggest_categorical(
#             "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
#         ),
#         "used_ram_limit": "3gb",
#     }

#     if param["bootstrap_type"] == "Bayesian":
#         param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
#     elif param["bootstrap_type"] == "Bernoulli":
#         param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

#     clf = catb.CatBoostClassifier(**param)
#     clf.fit(X_train, y_train)

#     preds = clf.predict(X_valid)
#     pred_labels = np.rint(preds)
#     accuracy = accuracy_score(y_valid, pred_labels)
#     return accuracy

In [ ]:
# %%time
# study = optuna.create_study(direction="maximize")
# study.optimize(objectivecatb, n_trials=100, show_progress_bar=True)

In [ ]:
# new_paramcatb = study.best_params
# new_paramcatb
# # optuna.visualization.plot_optimization_history(study)
# # optuna.visualization.plot_slice(study)

In [ ]:
catb_param={'objective': 'CrossEntropy',
 'colsample_bylevel': 0.03380683955420984,
 'depth': 10,
  "learning_rate":0.01,
  "iterations":100,
 'boosting_type': 'Ordered',
 'bootstrap_type': 'Bernoulli',
 'subsample': 0.4051072206464257}

In [ ]:
lgbm_param = {'boosting_type': 'gbdt',
 'lambda_l1': 0.134746148489252,
 'lambda_l2': 0.10521615726990495,
 'colsample_bytree': 0.8,
 'bagging_fraction': 0.7,
 'feature_fraction': 0.6,
 'learning_rate': 0.01,
 'max_depth': 2,
  "num_iterations":100,
 'num_leaves': 23,
 'min_child_samples': 46}

In [ ]:
# # lgbm_param=new_paramlgbm
# xgb_param=new_paramxgb
# catb_param=new_paramcatb

In [ ]:
# lgbm_param = {'boosting_type': 'gbdt',
#  'lambda_l1': 0.134746148489252,
#  'lambda_l2': 0.10521615726990495,
#  'colsample_bytree': 0.8,
#  'bagging_fraction': 0.7,
#  'feature_fraction': 0.6,
#  'learning_rate': 0.001019958066572347,
#  'max_depth': 9,
#  'num_leaves': 23,
#  'min_child_samples': 46}

# xgb_param = {'max_depth': 6,
#  'lambda': 4.5909601357645e-06,
#  'alpha': 0.011683603658842521,
#  'subsample': 0.6631109810494799,
#  'colsample_bytree': 0.3964625968646961,
#  'n_estimators': 16,
#  'min_child_weight': 11}

# catb_param = {'objective': 'CrossEntropy',
#  'colsample_bylevel': 0.040629495970693076,
#  'depth': 11,
#  'boosting_type': 'Plain',
#  'bootstrap_type': 'Bernoulli',
#  'subsample': 0.7446137182350138}

In [ ]:
#training

In [ ]:
N_SPLITS = 80  # previous:300, increasing N_SPLITS to remove error due to randomness

lgbm_preds = []
xgb_preds = []
catb_preds = []

prob = []
prob_train=[]

folds = StratifiedKFold(n_splits=N_SPLITS, shuffle=True)

for fold, (train_id, test_id) in enumerate(tqdm(folds.split(X, y), total=N_SPLITS)):
    X_train = X.iloc[train_id]
    y_train = y.iloc[train_id]
    X_valid = X.iloc[test_id]
    y_valid = y.iloc[test_id]

    lgbm_model = lgbm.LGBMClassifier(**lgbm_param)
    xgb_model  = xgb.XGBClassifier(**xgb_param)
    catb_model = catb.CatBoostClassifier(**catb_param, verbose=0)

    lgbm_model.fit(X_train, y_train)
    xgb_model.fit(X_train, y_train)
    catb_model.fit(X_train, y_train)



#     my_print(f'fold {fold + 1}')
#     my_print(f'Training Accuracy   :-  {(xgb_model.score(X_train, y_train)*100).round(2)}% | {(catb_model.score(X_train, y_train)*100).round(2)}%')
#     my_print(f'Validation Accuracy :-  {(xgb_model.score(X_valid, y_valid)*100).round(2)}% | {(catb_model.score(X_valid, y_valid)*100).round(2)}%')

#     prob1, prob2 = xgb_model.predict_proba(X_test), catb_model.predict_proba(X_test)
#     prob.append((prob1 + prob2) / 2)
#     prob11, prob21 = xgb_model.predict_proba(X), catb_model.predict_proba(X)
#     prob_train.append((prob11 + prob21) / 2)
# my_print('Model Trained !!!')

    my_print(f'fold {fold + 1}')
    my_print(f'Training Accuracy   :- {(lgbm_model.score(X_train, y_train)*100).round(2)}% | {(xgb_model.score(X_train, y_train)*100).round(2)}% | {(catb_model.score(X_train, y_train)*100).round(2)}%')
    my_print(f'Validation Accuracy :- {(lgbm_model.score(X_valid, y_valid)*100).round(2)}% | {(xgb_model.score(X_valid, y_valid)*100).round(2)}% | {(catb_model.score(X_valid, y_valid)*100).round(2)}%')

    prob1, prob2, prob3 = lgbm_model.predict_proba(X_test), xgb_model.predict_proba(X_test), catb_model.predict_proba(X_test)
    prob.append((prob1 + prob2 + prob3) / 3)
    prob11, prob21, prob31 = lgbm_model.predict_proba(X), xgb_model.predict_proba(X), catb_model.predict_proba(X)
    prob_train.append((prob11 + prob21 + prob31) / 3)
my_print('Model Trained !!!')

  0%|          | 0/80 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [ ]:
final = [[0,0]]
for i in range(N_SPLITS):
    final = final + prob[i]

final = final/N_SPLITS

In [ ]:
y_pred = pd.Series([np.argmax([i]) for i in final])

In [ ]:
y_pred_proba=pd.Series([max(i) for i in final])

In [ ]:
final = [[0,0]]
for i in range(N_SPLITS):
    final = final + prob_train[i]

final = final/N_SPLITS

In [ ]:
x_pred = pd.Series([np.argmax([i]) for i in final])

In [ ]:
x_pred_proba=pd.Series([max(i) for i in final])

In [ ]:
# test_data=test_data2

In [ ]:
train_data['y_pred_01'] = x_pred
test_data['y_pred_01'] = y_pred

In [ ]:
test_data['win_pred_score'] = y_pred_proba
train_data['win_pred_score'] = x_pred_proba

In [ ]:
train_data['win_pred_score'] = np.where( (train_data['y_pred_01']==0), (1-train_data['win_pred_score']), train_data['win_pred_score'])
test_data['win_pred_score'] = np.where( (test_data['y_pred_01']==0), (1-test_data['win_pred_score']), test_data['win_pred_score'])

In [ ]:
train_data['win_pred_team_id'] = np.where( (train_data['y_pred_01']==0), (train_data['team1_id']), train_data['team2_id'])
test_data['win_pred_team_id'] = np.where( (test_data['y_pred_01']==0), (test_data['team1_id']), test_data['team2_id'])

In [ ]:
df_feat_importance = pd.DataFrame({'feat_name':X.columns.tolist(), 'model_feat_imp_train':((xgb_model.feature_importances_+catb_model.feature_importances_)/2)}).sort_values(by='model_feat_imp_train', ascending=False)\
                                                                                                                        .reset_index(drop=True).head(10)
df_feat_importance

,feat_name,model_feat_imp_train
0,team_count_50runs_last15,16.655472
1,team_count_2wickets_last15,11.256953
2,team_count_sub20eco_last15,9.967546
3,team_winp_ground,2.672718
4,ground_avg_runs_last15,2.019522
5,toss_winner_01,2.005107
6,toss_decision_01,1.625302
7,team_winp_last5,1.462444
8,team_win_amount,1.190529
9,team1only_avg_runs_last15,1.139319


In [ ]:
train_data['dataset_type'] = 'train'
test_data['dataset_type'] = 'r1'

In [ ]:
## refactor

df_file1 = pd.concat([test_data[['match id','dataset_type','win_pred_team_id','win_pred_score',] + list(df_feat_importance['feat_name'].head(10))], \
                     train_data[['match id','dataset_type','win_pred_team_id','win_pred_score',] + list(df_feat_importance['feat_name'].head(10))]])

renaming_dict = {}
for i,col in enumerate(list(df_feat_importance['feat_name'].head(10))):
    renaming_dict[col] = f'indep_feat_id{i+1}'
df_file1.rename(columns=renaming_dict, inplace=True)

for i in range(1,11):
    if f'indep_feat_id{i}' not in df_file1.columns:
        df_file1[f'indep_feat_id{i}'] = np.nan

df_file1['train_algorithm'] = "xgboost;lgbm;catboost"
df_file1['is_ensemble'] = "yes"
df_file1['train_hps_trees'] = "100;100;100"
df_file1['train_hps_depth'] = "15;2;10"
df_file1['train_hps_lr'] = "0.01;0.01;0.01"

In [ ]:
df_file1.shape
df_file1.head()

,match id,dataset_type,win_pred_team_id,win_pred_score,indep_feat_id1,indep_feat_id2,indep_feat_id3,indep_feat_id4,indep_feat_id5,indep_feat_id6,indep_feat_id7,indep_feat_id8,indep_feat_id9,indep_feat_id10,train_algorithm,is_ensemble,train_hps_trees,train_hps_depth,train_hps_lr
0,9272619,r1,33928,0.513014,0.666667,1.294118,0.878049,0.673203,154.115385,1,0,1.000000,8.588235,167.933333,xgboost;lgbm;catboost,yes,100;100;100,15;2;10,0.01;0.01;0.01
1,9086958,r1,69,0.539936,0.142857,1.000000,0.706667,1.000000,142.833333,1,0,0.207921,1.000000,141.888889,xgboost;lgbm;catboost,yes,100;100;100,15;2;10,0.01;0.01;0.01
2,9433654,r1,9701,0.521720,0.666667,0.607143,0.825397,0.434229,155.733333,1,0,0.344262,0.980198,178.800000,xgboost;lgbm;catboost,yes,100;100;100,15;2;10,0.01;0.01;0.01
3,9097248,r1,22763,0.523690,0.500000,1.166667,1.611111,1.000000,142.500000,0,1,0.259259,0.271186,138.000000,xgboost;lgbm;catboost,yes,100;100;100,15;2;10,0.01;0.01;0.01
4,9097234,r1,23750,0.579128,0.166667,0.466667,0.526316,1.000000,NaN,1,0,1.000000,2.458333,155.800000,xgboost;lgbm;catboost,yes,100;100;100,15;2;10,0.01;0.01;0.01


#### File 2

In [ ]:
feature_desc = {'team_count_50runs_last15':'Ratio of number of 50s by players in team1 to number of 50s by players in team2 in last 15 games',\
               'team_winp_last5':'Ratio of team1\'s win % to team2\'s win % in last 5 games',\
               'ground_avg_runs_last15':'average runs scored in the ground in last 15 games',\
               'team1_winp_team2_last15':'Team1\'s win percentage against Team2 in last 15 games',\
               'team1only_avg_runs_last15':'team1\'s avg inning runs in last 15 games',\

               'season_num' : 'Numerical form of season. Takes 1 for oldest season and increases for latest seasons.',\
               'toss_winner_01' : 'Toss winner to numerical - 1 if team2 wins, else 0',\
               'toss_decision_01' : 'Toss decision - categorical - 1 if winner bats, 0 otherwise'}

In [ ]:
# df_feat_importance.rename(index={0:'feat_id'}, inplace=True)
df_file2 = df_feat_importance
df_file2['feat_id'] = [i+1 for i in df_file2.index]
df_file2['feat_rank_train'] = [i+1 for i in df_file2.index]
df_file2 = df_file2.set_index('feat_id')
df_file2['feat_description'] = df_file2['feat_name'].map(feature_desc)

In [ ]:
df_file2

,feat_name,model_feat_imp_train,feat_rank_train,feat_description
feat_id,,,,
1,team_count_50runs_last15,16.655472,1,Ratio of number of 50s by players in team1 to ...
2,team_count_2wickets_last15,11.256953,2,NaN
3,team_count_sub20eco_last15,9.967546,3,NaN
4,team_winp_ground,2.672718,4,NaN
5,ground_avg_runs_last15,2.019522,5,average runs scored in the ground in last 15 g...
6,toss_winner_01,2.005107,6,"Toss winner to numerical - 1 if team2 wins, el..."
7,toss_decision_01,1.625302,7,Toss decision - categorical - 1 if winner bats...
8,team_winp_last5,1.462444,8,Ratio of team1's win % to team2's win % in las...
9,team_win_amount,1.190529,9,NaN


## Save

In [ ]:
df_file1.to_csv('file1.csv', index=False)
df_file2.to_csv('file2.csv')